This notebook:

transformational variability analysis for context (sample 1 vs sample 2)


In [ ]:
!pip install dpca

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
%cd /content/gdrive/My Drive/dpca_code_V2
import numpy as np
import io
import pandas as pd
import matplotlib.pyplot as plt
import dpca as dp
from scipy import io
from scipy import stats
import scipy.io as sio
import math


In [ ]:
def center(data):
    """
    Center data in an S x N x K matrix
    by average over all S and N indices
    """
    return data - np.vstack(data).mean(0)

# TVI analysis

In [ ]:
%cd /content/gdrive/My Drive/roi_new_with_cue1_type
#conditions=[PMI_ori;UMI_ori;PMI_loc;UMI_loc;Cue1;ori_1,ori_2,loc_1,loc_2,cue_switch,PMI_ori_2,UMI_ori_2,PMI_loc_2,UMI_loc_2];
num_sub = 13
sublist = ['02','03','04','07','08','09','10','11','12','16','17','18','19']
r_spearman = []
p_spearman = []
dist_lower = []
dist_higher = []
for sub in range(num_sub):
    matdata2 = sio.loadmat(f'/content/gdrive/My Drive/roi_new_with_cue1_type/POCONN4_{sublist[sub]}_all_trials_delay2_500_roi18to23.mat')
    filename = f'POCONN4_{sublist[sub]}_all_trials_delay2_500_roi25.mat'
    matdata = sio.loadmat(filename) #trial_pat(324x500x20), conditions, angle_error1, angle_error2

    #S x T x N for data format
    trial_pat = matdata['trial_pat']
    conditions = matdata2['conditions'] #note this is matdata2
    angle_error1 = matdata2['angle_error1']
    angle_error2 = matdata2['angle_error2']

    ori_stim = np.unique(conditions[5,:])
    ori_stim1 = conditions[5,:]
    ori_stim2 = conditions[6,:]

    stim1_avg = np.zeros((len(ori_stim),trial_pat.shape[2], trial_pat.shape[1]))
    stim2_avg = np.zeros((len(ori_stim),trial_pat.shape[2], trial_pat.shape[1]))

    for i, iori in enumerate(ori_stim):
        iepochs = np.argwhere(ori_stim1 == iori)
        stim1_avg[i] = np.squeeze(trial_pat[iepochs, :, :].mean(0).T)

        iepochs = np.argwhere(ori_stim2 == iori)
        stim2_avg[i] = np.squeeze(trial_pat[iepochs, :, :].mean(0).T)

    TRs = np.array((4,5,6))

    # For Sample 1 subspace
    # stim1_tmp = center(stim1_avg[:,TRs,:])
    # stim1_model = dp.dPCA(stim1_tmp, n_dim = 2, old_version=True)
    # d = stim1_model.decoder

    # For Sample 2 subspace
    stim2_tmp = center(stim2_avg[:,TRs,:])
    stim2_model = dp.dPCA(stim2_tmp, n_dim = 2, old_version=True)
    d = stim2_model.decoder

    Xstim1 = np.matmul(center(stim1_avg),d)
    Xstim2 = np.matmul(center(stim2_avg),d)

    xystim1 = Xstim1[:,TRs,:].mean(1)
    xystim2 = Xstim2[:,TRs,:].mean(1)

    normdist = np.zeros((9))

    for iori in range(9):
        normdist[iori] = math.sqrt(((xystim1[iori,0]-xystim2[iori,0])**2)+((xystim1[iori,1]-xystim2[iori,1])**2))

    Xindivtrials = np.matmul(center(trial_pat.transpose((0,2,1))),d)
    num_trials = len(Xindivtrials)

    trdist = []
    trnormdist = []

    xygen = xystim2 #For Sample 2 subspace
    origen = ori_stim2
    for tr in range(num_trials):
          xytrial = Xindivtrials[tr,TRs,:].mean(0)
          trdist += [math.sqrt(((xytrial[0]-xygen[origen[tr]-1,0])**2)+((xytrial[1]-xygen[origen[tr]-1,1])**2))]
          trnormdist += [trdist[-1] / normdist[origen[tr]-1]]

    angle_error_abs = np.squeeze(abs(angle_error1)) #select which error to use
    trnormdist = np.array(trnormdist)
    sorted_dist = trnormdist[angle_error_abs.argsort()]
    dist_lower = np.append(dist_lower, np.mean(sorted_dist[:round(num_trials/2)]))
    dist_higher = np.append(dist_higher, np.mean(sorted_dist[round(num_trials/2):]))

# Calculate the correlation between TVI and error for each subject
#     r,p = stats.spearmanr(np.array(trnormdist),np.squeeze(abs(angle_error2)), alternative = 'greater')
#     r_spearman = np.append(r_spearman,round(r,2))
#     p_spearman = np.append(p_spearman,round(p,3))
#     print(sub)

# print(r_spearman)
# print(p_spearman)
# print(sum(p_spearman<0.05))

################# bar plot of transformational efficacy in Sample 1 space by response accuracy
A_lower = np.mean(dist_lower)
A_lower_err = stats.sem(dist_lower)
A_higher = np.mean(dist_higher)
A_higher_err = stats.sem(dist_higher)
print('lower M:', np.round(A_lower,2), 'higher M:', np.round(A_higher,2))
print('lower SEM:', np.round(A_lower_err,2), 'higher SEM:', np.round(A_higher_err,2))
xvals = [1,2]
yvals = [A_lower, A_higher]
error = [A_lower_err, A_higher_err]
accuracies = ['low error','high error']
fig, ax = plt.subplots()

ax.bar(xvals, yvals, yerr=error, align='center', alpha=0.5, ecolor='b', capsize=10)
ax.set_ylabel('Stim1 subspace distance index')
ax.set_xticks(xvals)
ax.set_xticklabels(accuracies)
ax.yaxis.grid(False)
ax.set_title('FEF Recall 1 Stim2 subspace')

stats.ttest_rel(dist_lower,dist_higher,alternative='less')